In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X[:3]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [2]:
y[:3]

0    0
1    0
2    0
Name: target, dtype: int64

In [3]:
from hypergbm.search_space import search_space_general
from hypergbm import HyperGBM, CompeteExperiment
from hypernets.searchers.evolution_searcher import EvolutionSearcher
from sklearn.metrics import get_scorer

rs = EvolutionSearcher(search_space_general,  200, 100, optimize_direction='max')
hk = HyperGBM(rs, task='multiclass', reward_metric='accuracy', callbacks=[])

experiment = CompeteExperiment(hk, X_train, y_train, X_test, y_test, callbacks=[], scorer=get_scorer('accuracy'),
                               feature_reselection=True,  # enable feature importance selection
                               feature_reselection_estimator_size=3,  # use 3 estimators to evaluate feature importance
                               feature_reselection_threshold=0.01)  # importance less than the threshold will not be selected

pipeline = experiment.run(max_trials=10)
pipeline

Pipeline(steps=[('data_clean',
                 DataCleanStep(cv=True,
                               data_cleaner_args={'correct_object_dtype': True,
                                                  'drop_columns': None,
                                                  'drop_constant_columns': True,
                                                  'drop_duplicated_columns': False,
                                                  'drop_idness_columns': True,
                                                  'drop_label_nan_rows': True,
                                                  'int_convert_to': 'float',
                                                  'nan_chars': None,
                                                  'reduce_mem_usage': False,
                                                  'reserve_columns': None},
                               name='data_clean')),
                ('fea...
                 GreedyEnsemble(weight=[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [4]:
import numpy as np
y_pred = pipeline.predict(X_test).astype(np.float64)

from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

1.0